In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types
import warnings
warnings.filterwarnings("ignore")

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
        .appName('test') \
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/21 16:29:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/raw/green/*/*')
df_green.registerTempTable('green')

In [4]:
df_green.rdd.take(10)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2019, 12, 18, 15, 52, 30), lpep_dropoff_datetime=datetime.datetime(2019, 12, 18, 15, 54, 39), store_and_fwd_flag='N', RatecodeID=1.0, PULocationID=264, DOLocationID=264, passenger_count=5.0, trip_distance=0.0, fare_amount=3.5, extra=0.5, mta_tax=0.5, tip_amount=0.01, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=4.81, payment_type=1.0, trip_type=1.0, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 1, 0, 45, 58), lpep_dropoff_datetime=datetime.datetime(2020, 1, 1, 0, 56, 39), store_and_fwd_flag='N', RatecodeID=5.0, PULocationID=66, DOLocationID=65, passenger_count=2.0, trip_distance=1.28, fare_amount=20.0, extra=0.0, mta_tax=0.0, tip_amount=4.06, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=24.36, payment_type=1.0, trip_type=2.0, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 1, 0, 41, 38

In [6]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount')\
        .rdd

In [14]:
from datetime import datetime

start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [16]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [17]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [19]:
from collections import namedtuple

RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [20]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [21]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [22]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [23]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-01 00:00:00|  66|            386.75|   18|
|2020-01-01 00:00:00| 210|              18.1|    2|
|2020-01-01 00:00:00|  82| 305.1200000000001|   28|
|2020-01-01 00:00:00|  74| 317.0900000000001|   24|
|2020-01-01 00:00:00| 134| 69.05000000000001|    6|
|2020-01-01 00:00:00| 116|328.76000000000005|   18|
|2020-01-01 00:00:00| 264|122.64999999999999|    4|
|2020-01-01 00:00:00|  42| 799.7600000000002|   52|
|2020-01-01 00:00:00| 166| 504.0100000000001|   33|
|2020-01-01 00:00:00| 256|296.62000000000006|   13|
|2020-01-01 00:00:00|  52|             83.33|    4|
|2020-01-01 00:00:00| 260|296.25000000000006|   21|
|2020-01-01 00:00:00|  22|              15.8|    1|
|2020-01-01 00:00:00| 112|312.26000000000005|   18|
|2020-01-01 00:00:00|  24|              87.6|    3|
|2020-01-01 00:00:00| 244|183.57999999999998|   12|
|2020-01-01 